In [38]:
import random
import re
from collections import defaultdict

def is_variable(pat):  #判断是否是?开头的单一变量
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

fail=[True,None]   #返回所有多变量和对应值的元组列表
def pat_match_with_seg(pattern,saying):
    if not pattern or not saying: return []
    pat=pattern[0]
    if is_variable(pat):
        return [(pat,saying[0])]+pat_match_with_seg(pattern[1:],saying[1:])
    elif is_pattern_segment(pat):
        match,index=segment_match(pattern,saying)
        return [match]+pat_match_with_seg(pattern[1:],saying[index:])
    elif pat==saying[0]:
        return pat_match_with_seg(pattern[1:],saying[1:])
    else:
        return fail

def segment_match(pattern,saying):    #返回多变量和对应值的元组，以及对应值中单词的个数，从后往前找
    seg_pat,rest=pattern[0],pattern[1:]
    seg_pat=seg_pat.replace('?*','?')
    if not rest: 
        return (seg_pat,saying), len(saying)
    for i, token in enumerate(saying):
        if rest[0]==token and is_match(rest[1:],saying[(i+1):]):
            return (seg_pat,saying[:i]),i
    return (seg_pat,saying),len(saying)
def is_match(rest,saying):    #判断非变量部分是否相同
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] !=saying[0]:
        return False
    return is_match(rest[1:],saying[1:])
            
def pat_to_dict(patterns):   #将变量与取值生成字典
    return {k: ' '.join(v) if isinstance(v,list) else v for k, v in patterns}
def substitute(rule, parsed_rules):    #替换变量部分
    if not rule: return []
    return [parsed_rules.get(rule[0], rule[0])] + substitute(rule[1:], parsed_rules)


In [39]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['which color do you need for ?y', 'Please let me know the color of ?y'],
    '?*x the color I need is ?*y': ['Do you confirm ?y is your choice, Yes to confirm, No to cancel', 'Please reconfirm your choice: ?y Yes to confirm, No to cancel'],
    'Yes': ['Comfirmed', 'Order completed', 'Please comment if you have other requirements'],
    'No': ['Order canceled', 'Thank you for using', 'Here are other goods you may be interested in'],
}

In [40]:
from functools import reduce
def remove_variable(sentence):
    sentence_combine=[]
    sentence_comp=[]
    for comp in sentence.split():
        if (not is_pattern_segment(comp)) and (not is_variable(comp)):
            sentence_comp +=[comp]
        else:
            sentence_combine.append(' '.join(sentence_comp))
            sentence_comp=[]
        if '' in sentence_combine: sentence_combine.remove('')
    return sentence_combine
def get_response(saying):
    for i,key in enumerate(rule_responses):
        if all(re.findall(x,saying) for x in remove_variable(key)):
            answer_pattern=random.choice(rule_responses[key])
            answer=' '.join(substitute(answer_pattern.split(),pat_to_dict(pat_match_with_seg(key.split(),saying.split()))))
            if i<=1:
                next_question=reduce(lambda x,y: x+y+'__',remove_variable(list(enumerate(rule_responses))[i+1][1]))
                saying=input(answer + '\nPlease provide more details. e.g. '+ next_question + '__')
            elif i==2:
                saying=input(answer + '\nPlease provide more details. e.g. Yes/No')
            else:
                return answer
            return get_response(saying)
    return 'System error'
    

In [41]:
get_response('hello')

How do you do
Please provide more details. e.g. I want__I want a bike
Please let me know the color of a bike
Please provide more details. e.g. the color I need is__the color I need is red
Do you confirm red is your choice, Yes to confirm, No to cancel
Please provide more details. e.g. Yes/NoYes


'Please comment if you have other requirements'

问题3:多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

Ans:相较基础代码，基于非变量部分是否一致来判定输入语句应对应哪一类回答；加入input函数，如用户按照提示输入语句，将按照rule_responses排列顺序交互回答。

问题4：
1.这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？ 

(1)输入的提问的词组必须与rule_responses中的pattern完全一致，可以增加同义词字典，自动将输入词组替换为同义词组后再与rule_responses匹配

(2)目前的输入识别基于非变量部分词组是否一致，若输入语句更长，或者系统需要识别的pattern增加，可能出现2条不同的输入语句包含同样的词组，从而导致输入类别识别错误，可考虑变量及非变量词组出现顺序对语句进行切割，实现更准确分类。

2.什么是数据驱动？数据驱动在这个程序里如何体现？
3.数据驱动与 AI 的关系是什么？ 
